In [1]:
import numpy as np
from collections import defaultdict
import itertools

In [2]:
# input = """AAAA
# BBCD
# BBCC
# EEEC"""

# input = """OOOOO
# OXOXO
# OOOOO
# OXOXO
# OOOOO"""

# input = """RRRRIICCFF
# RRRRIICCCF
# VVRRRCCFFF
# VVRCCCJFFF
# VVVVCJJCFE
# VVIVCCJJEE
# VVIIICJJEE
# MIIIIIJJEE
# MIIISIJEEE
# MMMISSJEEE"""

# input = """EEEEE
# EXXXX
# EEEEE
# EXXXX
# EEEEE"""

# input = """AAAAAA
# AAABBA
# AAABBA
# ABBAAA
# ABBAAA
# AAAAAA"""

# input = """RRRRIICCFF
# RRRRIICCCF
# VVRRRCCFFF
# VVRCCCJFFF
# VVVVCJJCFE
# VVIVCCJJEE
# VVIIICJJEE
# MIIIIIJJEE
# MIIISIJEEE
# MMMISSJEEE"""

input = open("inputs/12").read()

In [3]:
def parse_board(input):
    return np.array(list(map(list, input.splitlines())))


def add_tuple(t1, t2):
    return tuple(x + y for x, y in zip(t1, t2))


def mult_tuple(s, t):
    return tuple(x * s for x in t)


def in_bounds(location, bounds):
    return all(0 <= x < y for x, y in zip(location, bounds))


def chunk_contiguous_numbers(numbers):
    sorted_numbers = sorted(numbers)
    enumerated = enumerate(sorted_numbers)
    """
    Numbers:  3, 4, 5, 10, 11, 12
    Indices:  0, 1, 2,  3,  4,  5
    Differences: 3-0=3, 4-1=3, 5-2=3, 10-3=7, 11-4=7, 12-5=7
    """

    # trick: contiguous numbers will have the same difference between the index and their number
    chunks = [
        [x[1] for x in group]
        for _, group in itertools.groupby(enumerated, lambda x: x[1] - x[0])
    ]

    return chunks


dirs = [
    (-1, 0),  # up
    (0, 1),  # right
    (1, 0),  # down
    (0, -1),  # left
]

board = parse_board(input)

In [4]:
def p1(board):
    unvisited = set(np.ndindex(board.shape))

    def account_for_region(region_start):
        region_letter = board[region_start]
        perimeter = 0
        area = 0

        def dfs(node):
            nonlocal area
            nonlocal perimeter

            if node not in unvisited:
                return

            area += 1
            unvisited.remove(node)

            for dir in dirs:
                new_pos = add_tuple(node, dir)

                if in_bounds(new_pos, board.shape) and board[new_pos] == region_letter:
                    dfs(new_pos)
                else:
                    perimeter += 1

        dfs(region_start)

        return perimeter * area

    sm = 0
    while unvisited:
        s = account_for_region(next(iter(unvisited)))
        sm += s

    return sm

In [5]:
p1(board)

1450422

In [6]:
def count_combined_sides(sides):
    """This is definitely the most confusing part of the problem.

    Tl;dr we have to group the edges by orientation but horizontal and vertical aren't sufficient (because of the diagonal touching issue mentioned in the problem statement)
    So you have to group by the exact _dir_ of the fence relative to the internal region, and then further group by the orthogonal dimension, and then finally count contiguous chunks (to find whole sides).
    """
    by_dir = defaultdict(list)

    for pos, dir in sides:
        by_dir[dir].append(pos)

    sm = 0
    for dir, dir_group in by_dir.items():
        changing_coord = int(dir[0] != 0)

        dir_grouped_grouped = defaultdict(list)
        for x, y in dir_group:
            dir_grouped_grouped[x if changing_coord else y].append(
                y if changing_coord else x
            )

        for potential_sides in dir_grouped_grouped.values():
            sm += len(chunk_contiguous_numbers(potential_sides))

    return sm


def p2(board):
    unvisited = set(np.ndindex(board.shape))

    def account_for_region(region_start):
        region_letter = board[region_start]
        area = 0
        side_locs = set()

        def dfs(node):
            nonlocal area

            if node not in unvisited:
                return

            area += 1
            unvisited.remove(node)

            for dir in dirs:
                new_pos = add_tuple(node, dir)

                if in_bounds(new_pos, board.shape) and board[new_pos] == region_letter:
                    dfs(new_pos)
                else:
                    side_locs.add((node, dir))

        dfs(region_start)
        perimeter = count_combined_sides(side_locs)
        return perimeter * area

    sm = 0
    while unvisited:
        unvisited_pos = next(iter(unvisited))
        s = account_for_region(unvisited_pos)
        sm += s

    return sm

In [7]:
p2(board)

906606